In [5]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_Movregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV2']=t_e['DEV2'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)
t_e=t_e.drop(['Unnamed: 0'],1)
t_e.info()

t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)

t_e.watched_time_y=(t_e.watched_time_y/60)
t_e['ts-f1']=t_e['ts-f1'].abs()
t_e.rating[t_e.watched_time_x > 125] = t_e.rating[t_e.watched_time_x > 125].fillna(9.0)# вооще дропнуть их надо бы

from collections import Counter
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')

coun.head()

t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

t_e=t_e.drop(['longfilm','shortfilm'],1)

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])
t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)

t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])

t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)


t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8447928 entries, 0 to 8447927
Data columns (total 33 columns):
element_uid       int32
user_uid          int32
watched_time_x    float64
rating            float64
duration          int16
feature_2         float64
feature_3         int16
feature_4         float64
quantity          int64
rating_mean       float64
CM_S              int16
CM_P              int16
CM_R              int16
feature_5_65      int16
feature_5_00      int16
feature_5_68      int16
feature_5_59      int16
feature_5_-1      int16
feature_5_44      int16
smallf3           int16
middlef3          int16
bigf3             int16
longfilm          int64
shortfilm         int64
ts-f1             float64
watched_time_y    int64
DEV0              int16
DEV1              int16
DEV2              int16
DEV3              int16
DEV4              int16
DEV5              int16
DEV6              int16
dtypes: float64(6), int16(21), int32(2), int64(4)
memory usage: 1.0 GB


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
t_e=t_e.drop(['user_rating_mean'],1)
transactions_exp=t_e
transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 
X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating


In [3]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

t_e=t_e.drop(['user_rating_mean'],1)
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 
transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp=transactions_exp.drop(['worst'],1)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)

X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating




D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:


estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )    
estimator.fit(X_train, y_train)
submit1 = pd.DataFrame(estimator.predict(X_test))


X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
    
    
    
    
    
    
    
    
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
OLD.to_csv("XGB17.03.csv")
print('end ARHIP')


end ARHIP


In [8]:
X_train=X_train.drop([
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349503 entries, 0 to 349502
Data columns (total 2 columns):
element_uid    349503 non-null int32
user_uid       349503 non-null int32
dtypes: int32(2)
memory usage: 2.7 MB


In [13]:

OLD2=pd.concat([X_train, y_train], 1)


KeyError: 0

In [16]:
OLD=OLD.append(OLD2)

In [17]:
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6921576 entries, 0 to 349502
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
rating         float64
dtypes: float64(1), int32(2)
memory usage: 158.4 MB


In [18]:

OLD.to_csv("XGB17.03.csv")

In [9]:
DEMO1=DEMO1.drop(['worst'], 1)

In [10]:

DEMOsub = pd.DataFrame(model2.predict(DEMO1))
DEMO1=DEMO1.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
DEMO1=DEMO1.reset_index()
DEMO1=DEMO1.drop(['index'], 1)
OLDd=pd.concat([DEMO1, DEMOsub], 1)
OLDd['rating']=OLDd[0]
OLDd=OLDd.drop([0],1)
OLDdem=OLD1.append(OLDd)
OLDdem.to_csv("MovRatingRFNEW3(doobuch)beznulltrain.csv")

X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)
OLD=OLD.append(X_train)
OLD.to_csv("MovRatingRFNEW3.csv")



del X_train, X_test, y_train, y_test, OLD, transactions_exp, t_e, coun
print('1')


import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
from pprint import pprint
pd.set_option('display.max_columns', 50)
DATA_PATH='D:\download'
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_Movregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV2']=t_e['DEV2'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)
t_e=t_e.drop(['Unnamed: 0'],1)
t_e.info()

t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)

t_e.watched_time_y=(t_e.watched_time_y/60)
t_e['ts-f1']=t_e['ts-f1'].abs()
t_e.rating[t_e.watched_time_x > 125] = t_e.rating[t_e.watched_time_x > 125].fillna(9.0)# вооще дропнуть их надо бы

from collections import Counter
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')

coun.head()

t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

t_e=t_e.drop(['longfilm','shortfilm'],1)

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])
t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)

t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])

t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)


t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)



import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

t_e=t_e.drop(['user_rating_mean'],1)
transactions_exp=t_e


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0) 
transactions_exp['worst']=transactions_exp.watched_time_x.map(lambda x:1 if (x < 0.2 and x >0.005) else 0)
transactions_exp[transactions_exp.worst == 1]=transactions_exp[transactions_exp.worst == 1].fillna(2.0)
transactions_exp=transactions_exp.drop(['worst'],1)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating





model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))



X_test=X_test.drop([
'watched_time_x',
'watched',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4',
'Q_us_views=1',
'Q_us_views=4'],1)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)

OLD=pd.concat([X_test, submit1], 1)
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)



OLD.to_csv("MovRatingRFNEW3(dobavitX_trainbezworst).csv")

1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8447928 entries, 0 to 8447927
Data columns (total 33 columns):
element_uid       int32
user_uid          int32
watched_time_x    float64
rating            float64
duration          int16
feature_2         float64
feature_3         int16
feature_4         float64
quantity          int64
rating_mean       float64
CM_S              int16
CM_P              int16
CM_R              int16
feature_5_65      int16
feature_5_00      int16
feature_5_68      int16
feature_5_59      int16
feature_5_-1      int16
feature_5_44      int16
smallf3           int16
middlef3          int16
bigf3             int16
longfilm          int64
shortfilm         int64
ts-f1             float64
watched_time_y    int64
DEV0              int16
DEV1              int16
DEV2              int16
DEV3              int16
DEV4              int16
DEV5              int16
DEV6              int16
dtypes: float64(6), int16(21), int32(2), int64(4)
memory usage: 1.0 GB


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
OLD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6572073 entries, 0 to 6572072
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
rating         float64
dtypes: float64(1), int32(2)
memory usage: 100.3 MB


In [12]:

del X_train, X_test, y_train, y_test, OLD, transactions_exp, t_e, coun

In [13]:
t_e = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_Movregress.csv'),
    
)
t_e['DEV0']=t_e['DEV0'].astype(np.int16)
t_e['DEV1']=t_e['DEV1'].astype(np.int16)
t_e['DEV2']=t_e['DEV2'].astype(np.int16)
t_e['DEV3']=t_e['DEV3'].astype(np.int16)
t_e['DEV4']=t_e['DEV4'].astype(np.int16)
t_e['DEV5']=t_e['DEV5'].astype(np.int16)
t_e['DEV6']=t_e['DEV6'].astype(np.int16)
t_e['feature_5_65']=t_e['feature_5_65'].astype(np.int16)
t_e['feature_5_00']=t_e['feature_5_00'].astype(np.int16)
t_e['feature_5_68']=t_e['feature_5_68'].astype(np.int16)
t_e['feature_5_59']=t_e['feature_5_59'].astype(np.int16)
t_e['feature_5_-1']=t_e['feature_5_-1'].astype(np.int16)
t_e['smallf3']= t_e['smallf3'].astype(np.int16)
t_e['middlef3']=t_e['middlef3'].astype(np.int16)
t_e['bigf3']=t_e['bigf3'].astype(np.int16)
t_e['CM_S']=t_e['CM_S'].astype(np.int16)
t_e['CM_P']=t_e['CM_P'].astype(np.int16)
t_e['CM_R']=t_e['CM_R'].astype(np.int16)
t_e['feature_5_44']=t_e['feature_5_44'].astype(np.int16)
t_e['feature_3']=t_e['feature_3'].astype(np.int16)
t_e['duration']=t_e['duration'].astype(np.int16)
t_e['element_uid']=t_e['element_uid'].astype(np.int32)
t_e['user_uid']=t_e['user_uid'].astype(np.int32)
t_e=t_e.drop(['Unnamed: 0'],1)
t_e.info()

t_e.duration= t_e.duration.map(lambda x: 5 if x == 0 else x)
t_e.watched_time_x=(t_e.watched_time_y/60)/(t_e.duration)

t_e.watched_time_y=(t_e.watched_time_y/60)
t_e['ts-f1']=t_e['ts-f1'].abs()
t_e.rating[t_e.watched_time_x > 125] = t_e.rating[t_e.watched_time_x > 125].fillna(9.0)# вооще дропнуть их надо бы

from collections import Counter
t_e["user_rating_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").rating.mean())
coun = pd.DataFrame.from_dict(Counter(t_e['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
t_e=pd.merge(t_e, coun, on='user_uid')

coun.head()

t_e["wt_y_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_y.mean())
t_e["wt_x_user_mean"] = t_e.user_uid.map(t_e.groupby("user_uid").watched_time_x.mean())
t_e["new_content"]=t_e['ts-f1'].map(lambda x: 1 if x<31 else 0)

t_e=t_e.drop(['longfilm','shortfilm'],1)

t_e['quartile__element_views'] = pd.qcut(t_e['quantity'], 4, ['1','2','3','4'])
t_e['quartile__user_views'] = pd.qcut(t_e['quantity_user_views'], 4, ['1','2','3','4'])
t_e['quartile__duration'] = pd.qcut(t_e['duration'], 4, ['1','2','3','4'])
t_e['quartile__wtx'] = pd.qcut(t_e['wt_x_user_mean'], 4, ['1','2','3','4'])
t_e["Q_el_views=1"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_el_views=4"]=t_e['quartile__element_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_us_views=1"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_us_views=4"]=t_e['quartile__user_views'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_duration=1"]=t_e['quartile__duration'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_duration=4"]=t_e['quartile__duration'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_wtx=1"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_wtx=4"]=t_e['quartile__wtx'].map(lambda x: 1 if x=='4' else 0)

t_e['quartile__f2'] = pd.qcut(t_e['feature_2'], 4, ['1','2','3','4'])
t_e['quartile__f4'] = pd.qcut(t_e['feature_4'], 4, ['1','2','3','4'])

t_e["Q_f2=1"]=t_e['quartile__f2'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f2=4"]=t_e['quartile__f2'].map(lambda x: 1 if x=='4' else 0)
t_e["Q_f4=1"]=t_e['quartile__f4'].map(lambda x: 1 if x=='1' else 0)
t_e["Q_f4=4"]=t_e['quartile__f4'].map(lambda x: 1 if x=='4' else 0)
t_e=t_e.drop(['quartile__element_views', 'quartile__user_views',  'quartile__duration', 'quartile__wtx', 'quartile__f2', 'quartile__f4'],1)


t_e['new_content']=t_e['new_content'].astype(np.int16)
t_e['Q_el_views=1']=t_e['Q_el_views=1'].astype(np.int16)
t_e['Q_el_views=4']=t_e['Q_el_views=4'].astype(np.int16)
t_e['Q_us_views=1']=t_e['Q_us_views=1'].astype(np.int16)
t_e['Q_us_views=4']=t_e['Q_us_views=4'].astype(np.int16)
t_e['Q_duration=1']=t_e['Q_duration=1'].astype(np.int16)
t_e['Q_duration=4']=t_e['Q_duration=4'].astype(np.int16)
t_e['Q_wtx=1']=t_e['Q_wtx=1'].astype(np.int16)
t_e['Q_wtx=4']=t_e['Q_wtx=4'].astype(np.int16)
t_e['Q_f2=1']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f2=4']=t_e['Q_f2=1'].astype(np.int16)
t_e['Q_f4=1']=t_e['Q_f4=1'].astype(np.int16)
t_e['Q_f4=4']=t_e['Q_f4=1'].astype(np.int16)



import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

t_e=t_e.drop(['user_rating_mean'],1)
transactions_exp=t_e

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8447928 entries, 0 to 8447927
Data columns (total 33 columns):
element_uid       int32
user_uid          int32
watched_time_x    float64
rating            float64
duration          int16
feature_2         float64
feature_3         int16
feature_4         float64
quantity          int64
rating_mean       float64
CM_S              int16
CM_P              int16
CM_R              int16
feature_5_65      int16
feature_5_00      int16
feature_5_68      int16
feature_5_59      int16
feature_5_-1      int16
feature_5_44      int16
smallf3           int16
middlef3          int16
bigf3             int16
longfilm          int64
shortfilm         int64
ts-f1             float64
watched_time_y    int64
DEV0              int16
DEV1              int16
DEV2              int16
DEV3              int16
DEV4              int16
DEV5              int16
DEV6              int16
dtypes: float64(6), int16(21), int32(2), int64(4)
memory usage: 1.0 GB


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
OO = pd.read_csv('MovRatingRFNEW3(doobuch)beznulltrain.csv')
main = pd.read_csv('MovRatingRFNEW3.csv')

In [15]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8447928 entries, 0 to 8447927
Data columns (total 4 columns):
Unnamed: 0     int64
element_uid    int64
user_uid       int64
rating         float64
dtypes: float64(1), int64(3)
memory usage: 257.8 MB


In [16]:
OO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8098425 entries, 0 to 8098424
Data columns (total 4 columns):
Unnamed: 0     int64
element_uid    int64
user_uid       int64
rating         float64
dtypes: float64(1), int64(3)
memory usage: 247.1 MB


In [19]:

OLD = pd.read_csv('MovRatingRFNEW3(dobavitX_trainbezworst).csv')
OLD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6572073 entries, 0 to 6572072
Data columns (total 4 columns):
Unnamed: 0     int64
element_uid    int64
user_uid       int64
rating         float64
dtypes: float64(1), int64(3)
memory usage: 200.6 MB


In [20]:


transactions_exp.rating_mean=transactions_exp.rating_mean.fillna(5.0)
transactions_exp['watched']=transactions_exp.watched_time_x.map(lambda x: 0 if x < 0.51 else 1)


X_train = transactions_exp[~transactions_exp.rating.isnull()]
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
X_train=X_train.drop(['watched',
'watched_time_x',
'duration',
'feature_2',
'feature_3',
'feature_4',
'quantity',
'rating_mean',
'CM_S',
'CM_P',
'CM_R',
'feature_5_65',
'feature_5_00',
'feature_5_68',
'feature_5_59',
'feature_5_-1',
'feature_5_44',
'smallf3',
'middlef3',
'bigf3',
'ts-f1',
'watched_time_y',
'DEV0',
'DEV1',
'DEV2',
'DEV3',
'DEV4',
'DEV5',
'DEV6',
'quantity_user_views',
'wt_y_user_mean',         
'wt_x_user_mean',
'new_content',
'Q_el_views=1',
'Q_el_views=4',
'Q_us_views=1',
'Q_us_views=4',
'Q_duration=1',
'Q_duration=4',
'Q_wtx=1',
'Q_wtx=4',
'Q_f2=1',
'Q_f2=4',
'Q_f4=1',
'Q_f4=4'],1)
X_train=X_train.reset_index()
X_train=X_train.drop(['index'], 1)


In [21]:
X_train.head()

,element_uid,user_uid,rating
0,3336,5177,9.0
1,2468,5177,9.0
2,6573,5177,4.0
3,8271,5177,8.0
4,6209,5177,8.0


In [24]:
OLD=OLD.drop(['Unnamed: 0'], 1)
OO=OO.drop(['Unnamed: 0'], 1)
OO=OO.append(X_train)

OLD=OLD.append(X_train)

In [25]:
OLD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6921576 entries, 0 to 349502
Data columns (total 3 columns):
element_uid    int64
user_uid       int64
rating         float64
dtypes: float64(1), int64(2)
memory usage: 211.2 MB


In [26]:
OO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8447928 entries, 0 to 349502
Data columns (total 3 columns):
element_uid    int64
user_uid       int64
rating         float64
dtypes: float64(1), int64(2)
memory usage: 257.8 MB


In [27]:

OLD.to_csv("MovRatingRFNEW3(dobavitX_trainbezworst).csv")
OO.to_csv("'MovRatingRFNEW3(doobuch)beznulltrain.csv'")